In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator

from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

import tensorflow as tf
import random
import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


labels.csv
sample_submission.csv
test
train



In [2]:
##############
# parameters #
##############
# dontFreezeLast = 0;

# patience = 10;

# gpuName = '/device:GPU:0'
# workers = 2;
# histogram_freq = 0;

# epochs = 100;
# validation_size=0.3;

modelPath = '../models/VGG16_opt/';
modelName = 'run2.h5';
os.environ["CUDA_VISIBLE_DEVICES"]="1"

if not os.path.exists(modelPath):
    os.makedirs(modelPath)

First we will read in the csv's so we can see some more information on the filenames and breeds

In [3]:
# df_train = pd.read_csv('../input/labels.csv')
# df_test = pd.read_csv('../input/sample_submission.csv')

# print('Training images: ',df_train.shape[0])
# print('Test images: ',df_test.shape[0])

# reduce dimensionality
#df_train = df_train.head(100)
#df_test = df_test.head(100)

In [4]:
#df_train.head(10)

We can see that the breed needs to be one-hot encoded for the final submission, so we will now do this.

In [5]:
# targets_series = pd.Series(df_train['breed'])
# one_hot = pd.get_dummies(targets_series, sparse = True)

In [6]:
# one_hot_labels = np.asarray(one_hot)

Next we will read in all of the images for test and train, using a for loop through the values of the csv files. I have also set an im_size variable which sets the size for the image to be re-sized to, 90x90 px, you should play with this number to see how it affects accuracy.

In [7]:
# im_size = 90

In [8]:
# x_train = []
# y_train = []
# x_test = []

In [9]:
# i = 0 
# for f, breed in tqdm(df_train.values[:10]):
#     img = cv2.imread('../input/train/{}.jpg'.format(f))
#     label = one_hot_labels[i]
#     x_train.append(cv2.resize(img, (im_size, im_size)))
#     y_train.append(label)
#     i += 1

In [10]:
# for f in tqdm(df_test['id'].values):
#     img = cv2.imread('../input/test/{}.jpg'.format(f))
#     x_test.append(cv2.resize(img, (im_size, im_size)))

In [11]:
# y_train_raw = np.array(y_train, np.uint8)
# x_train_raw = np.array(x_train, np.float32) / 255.
# x_test  = np.array(x_test, np.float32) / 255.

We check the shape of the outputs to make sure everyting went as expected.

In [12]:
# print(x_train_raw.shape)
# print(y_train_raw.shape)
# print(x_test.shape)

We can see above that there are 120 different breeds. We can put this in a num_class variable below that can then be used when creating the CNN model.

In [13]:
# num_class = y_train_raw.shape[1]
# print('Number of classes: ', num_class)

It is important to create a validation set so that you can gauge the performance of your model on independent data, unseen to the model in training. We do this by splitting the current training set (x_train_raw) and the corresponding labels (y_train_raw) so that we set aside 30 % of the data at random and put these in validation sets (X_valid and Y_valid).

* This split needs to be improved so that it contains images from every class, with 120 separate classes some can not be represented and so the validation score is not informative.

In [14]:
# X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=validation_size, random_state=1)

Now we build the CNN architecture. Here we are using a pre-trained model VGG19 which has already been trained to identify many different dog breeds (as well as a lot of other objects from the imagenet dataset see here for more information: http://image-net.org/about-overview). Unfortunately it doesn't seem possible to downlod the weights from within this kernel so make sure you set the weights argument to 'imagenet' and not None, as it currently is below.

We then remove the final layer and instead replace it with a single dense layer with the number of nodes corresponding to the number of breed classes we have (120).

In [15]:
def data():
    print('Getting data')
    df_train = pd.read_csv('../input/labels.csv')
    df_test = pd.read_csv('../input/sample_submission.csv')
    
    targets_series = pd.Series(df_train['breed'])
    one_hot = pd.get_dummies(targets_series, sparse = True)
    one_hot_labels = np.asarray(one_hot)
    
    im_size = 90;
    x_train = []
    y_train = []
    x_test = []
    
    i = 0 
    for f, breed in tqdm(df_train.values):
        img = cv2.imread('../input/train/{}.jpg'.format(f))
        label = one_hot_labels[i]
        x_train.append(cv2.resize(img, (im_size, im_size)))
        y_train.append(label)
        i += 1
    
    y_train_raw = np.array(y_train, np.uint8)
    x_train_raw = np.array(x_train, np.float32) / 255.
    num_class = y_train_raw.shape[1]
    
    print('Splitting into training/validation')
    X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=0.3, random_state=1)
    
    return X_train, Y_train, X_valid, Y_valid

In [ ]:
# prepare data and model for hyperas

def model(X_train,Y_train,X_valid,Y_valid):
    print('Creating model')
    base_model = VGG16(weights = 'imagenet',
                       include_top=False,
                       input_shape=(im_size, im_size, 3))

    dropout = 0.6;
    dontFreeze = 3;
    batchSize = 64;
    #
    momentum = {{uniform(0.95,0.9999999)}};
    
    print()
    print('dropout=',dropout)
    print('dontFreeze=',dontFreeze)
    print('batchSize=',batchSize)
    print('momentum=',momentum)
    print()
    
    stepsPerEpoch = round( len(X_train)/batchSize );
    
    # Add a new top layer
    x = base_model.output
    x = BatchNormalization(axis=1,momentum=momentum)(x)
    x = Flatten()(x)
    x = Dropout(dropout)(x)
    x = Dense(1024,activation='relu')(x)
    x = Dense(512,activation='relu')(x)
    # in any case:
    predictions = Dense(num_class, activation='softmax')(x)

    # This is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)

    # First: train only the top layers (which were randomly initialized)
    for i in range(len(base_model.layers)-dontFreeze):
        base_model.layers[i].trainable = False

    # predetermined optimizer
    lr=0.00020389590556056983;
    beta_1=0.9453158868247398;
    beta_2=0.9925872692991417;
    decay=0.000821336141287018;
    adam = keras.optimizers.Adam(lr=lr,beta_1=beta_1,beta_2=beta_2,decay=decay)
    model.compile(loss='categorical_crossentropy', 
                  optimizer=adam, 
                  metrics=['accuracy'])

    callbacks_list = [];
    callbacks_list.append(keras.callbacks.EarlyStopping(
        monitor='val_acc',
        patience=10,
        verbose=1));


    # data augmentation & fitting
    datagen = ImageDataGenerator(
        rotation_range=30,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.5,
        zoom_range=0.5,
        horizontal_flip=True,
        vertical_flip=True);
    
    model.fit_generator(
        datagen.flow(X_train,Y_train,batch_size=batchSize),
        steps_per_epoch=stepsPerEpoch,
        epochs=150,
        verbose=1,
        validation_data=(X_valid,Y_valid),
        workers=2,
        shuffle=True,
        callbacks=callbacks_list)
#     model.fit(X_train, Y_train,
#       epochs=100,
#       batch_size = batchSize,
#       validation_data=(X_valid, Y_valid),
#       verbose=1,
#       callbacks=callbacks_list)

    score, acc = model.evaluate(X_valid, Y_valid, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}


In [ ]:
best_run, best_model = optim.minimize(model=model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=3,
                                          trials=Trials(),
                                          notebook_name='VGG16Optimization_2')

>>> Imports:
#coding=utf-8

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from keras.applications import VGG16
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.layers import Dense, Dropout, Flatten, BatchNormalization
except:
    pass

try:
    from keras.preprocessing.image import ImageDataGenerator
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    import random
except:
    pass

try:
    import os
except:
    pass

try:
    from tqdm import tqdm
except:
    pass

try:
    from sklearn import preprocessing
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    import c

100%|██████████| 10222/10222 [00:37<00:00, 269.50it/s]


Splitting into training/validation
Creating model

dropout= 0.6
dontFreeze= 3
batchSize= 64
momentum= 0.9718557692599662

Epoch 1/150
112/112 [==============================] - 38s 338ms/step - loss: 4.9514 - acc: 0.0101 - val_loss: 4.7911 - val_acc: 0.0085
Epoch 2/150
112/112 [==============================] - 24s 214ms/step - loss: 4.8072 - acc: 0.0131 - val_loss: 4.7580 - val_acc: 0.0170
Epoch 3/150
112/112 [==============================] - 24s 215ms/step - loss: 4.7428 - acc: 0.0149 - val_loss: 4.6323 - val_acc: 0.0248
Epoch 4/150
112/112 [==============================] - 24s 217ms/step - loss: 4.6579 - acc: 0.0213 - val_loss: 4.5765 - val_acc: 0.0323
Epoch 5/150
112/112 [==============================] - 24s 217ms/step - loss: 4.5595 - acc: 0.0315 - val_loss: 4.4165 - val_acc: 0.0424
Epoch 6/150
112/112 [==============================] - 24s 217ms/step - loss: 4.4561 - acc: 0.0365 - val_loss: 4.2612 - val_acc: 0.0551
Epoch 7/150
112/112 [==============================] - 24s 217

Epoch 60/150
112/112 [==============================] - 25s 219ms/step - loss: 3.1324 - acc: 0.2233 - val_loss: 3.3068 - val_acc: 0.2074
Epoch 61/150
112/112 [==============================] - 24s 218ms/step - loss: 3.1345 - acc: 0.2228 - val_loss: 3.2549 - val_acc: 0.2116
Epoch 62/150
112/112 [==============================] - 25s 221ms/step - loss: 3.0927 - acc: 0.2367 - val_loss: 3.2773 - val_acc: 0.2168
Epoch 63/150
112/112 [==============================] - 25s 220ms/step - loss: 3.0969 - acc: 0.2358 - val_loss: 3.2399 - val_acc: 0.2172
Epoch 64/150
112/112 [==============================] - 25s 221ms/step - loss: 3.0978 - acc: 0.2331 - val_loss: 3.2674 - val_acc: 0.2136
Epoch 65/150
112/112 [==============================] - 25s 220ms/step - loss: 3.0754 - acc: 0.2357 - val_loss: 3.2653 - val_acc: 0.2145
Epoch 66/150
112/112 [==============================] - 25s 220ms/step - loss: 3.0576 - acc: 0.2481 - val_loss: 3.2967 - val_acc: 0.2149
Epoch 67/150
112/112 [===================

Epoch 14/150
112/112 [==============================] - 25s 220ms/step - loss: 3.9255 - acc: 0.0962 - val_loss: 3.8515 - val_acc: 0.1148
Epoch 15/150
112/112 [==============================] - 25s 220ms/step - loss: 3.8870 - acc: 0.1062 - val_loss: 3.7487 - val_acc: 0.1226
Epoch 16/150
112/112 [==============================] - 25s 220ms/step - loss: 3.8513 - acc: 0.1062 - val_loss: 3.6742 - val_acc: 0.1307
Epoch 17/150
112/112 [==============================] - 26s 228ms/step - loss: 3.8143 - acc: 0.1100 - val_loss: 3.6496 - val_acc: 0.1396
Epoch 18/150
112/112 [==============================] - 25s 220ms/step - loss: 3.7787 - acc: 0.1158 - val_loss: 3.6902 - val_acc: 0.1304
Epoch 19/150
112/112 [==============================] - 25s 221ms/step - loss: 3.7391 - acc: 0.1212 - val_loss: 3.6125 - val_acc: 0.1444
Epoch 20/150
112/112 [==============================] - 25s 220ms/step - loss: 3.7220 - acc: 0.1304 - val_loss: 3.6619 - val_acc: 0.1470
Epoch 21/150
112/112 [===================

Epoch 14/150
112/112 [==============================] - 24s 216ms/step - loss: 3.9101 - acc: 0.0983 - val_loss: 3.7231 - val_acc: 0.1161
Epoch 15/150
112/112 [==============================] - 24s 217ms/step - loss: 3.8688 - acc: 0.1047 - val_loss: 3.7216 - val_acc: 0.1161
Epoch 16/150
112/112 [==============================] - 24s 216ms/step - loss: 3.8323 - acc: 0.1080 - val_loss: 3.6790 - val_acc: 0.1255
Epoch 17/150
112/112 [==============================] - 24s 219ms/step - loss: 3.8072 - acc: 0.1145 - val_loss: 3.6271 - val_acc: 0.1350
Epoch 18/150
112/112 [==============================] - 25s 220ms/step - loss: 3.7958 - acc: 0.1133 - val_loss: 3.6653 - val_acc: 0.1363
Epoch 19/150
112/112 [==============================] - 25s 220ms/step - loss: 3.7366 - acc: 0.1221 - val_loss: 3.6064 - val_acc: 0.1350
Epoch 20/150
112/112 [==============================] - 25s 220ms/step - loss: 3.7288 - acc: 0.1262 - val_loss: 3.5868 - val_acc: 0.1467
Epoch 21/150
112/112 [===================

Epoch 74/150
112/112 [==============================] - 24s 216ms/step - loss: 3.0578 - acc: 0.2434 - val_loss: 3.3001 - val_acc: 0.2175
Epoch 75/150
112/112 [==============================] - 24s 217ms/step - loss: 3.0416 - acc: 0.2446 - val_loss: 3.2839 - val_acc: 0.2266
Epoch 76/150
112/112 [==============================] - 24s 217ms/step - loss: 3.0343 - acc: 0.2533 - val_loss: 3.2720 - val_acc: 0.2227
Epoch 77/150
112/112 [==============================] - 24s 216ms/step - loss: 3.0176 - acc: 0.2526 - val_loss: 3.2639 - val_acc: 0.2269
Epoch 78/150
112/112 [==============================] - 25s 225ms/step - loss: 2.9971 - acc: 0.2550 - val_loss: 3.2732 - val_acc: 0.2230
Epoch 79/150
112/112 [==============================] - 24s 217ms/step - loss: 3.0064 - acc: 0.2537 - val_loss: 3.2471 - val_acc: 0.2299
Epoch 80/150
112/112 [==============================] - 24s 217ms/step - loss: 3.0119 - acc: 0.2546 - val_loss: 3.2716 - val_acc: 0.2266
Epoch 81/150
112/112 [===================

In [ ]:
X_train, Y_train, X_test, Y_test = data()
val_loss, val_acc = best_model.evaluate(X_test, Y_test);
print("Evalutation of best performing model:")
print("Validation loss: ", val_loss)
print("Validation accuracy: ", val_acc)

In [ ]:
best_model.save(modelPath+modelName);
best_model.summary()

Remember, accuracy is low here because we are not taking advantage of the pre-trained weights as they cannot be downloaded in the kernel. This means we are training the wights from scratch and I we have only run 1 epoch due to the hardware constraints in the kernel.

Next we will make our predictions.

In [ ]:
# preds = model.predict(x_test, verbose=1)

In [ ]:
# sub = pd.DataFrame(preds)
# # Set column names to those generated by the one-hot encoding earlier
# col_names = one_hot.columns.values
# sub.columns = col_names
# # Insert the column id from the sample_submission at the start of the data frame
# sub.insert(0, 'id', df_test['id'])
# sub.head(10)